# Manual Cuts
This notebook explores some manual cuts we can make on the data based on analysis, performed below, on features distinguishing signal from background. We use some educated guessing as to what variables might be distinguishing but also some physically motivated cuts which will be explained as and when they are performed.

## Imports and Constants
Firstly we quickly import all the modules we are going to need during this analysis. The utilities module contains all our classes that can be used for data manupulation and calling in constants.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from utilities import Data, Consts, Cut

In [3]:
RFNAME, RSUFFIX = Consts().get_real_tuple()
SFNAME, SSUFFIX = Consts().get_simulated_tuple()
PARTICLES = ["p", "K", "L1", "L2"]

## Create Data & Cut Objects
Now we have the file paths and decay trees we wish to analyse, that is those of the real blinded LHCb data and Monte-Carlo simulated signal, we can construct some Data objects which will keep track of these data as cuts are performed. We also create a Cut object which will be used to calculate the cut paramters for some given cuts and then apply them to multiple Data objects.

In [4]:
real, sim = Data(RFNAME, RSUFFIX), Data(SFNAME, SSUFFIX)
rcut = Cut(real)
scut = Cut(sim)

## Apply Different Cuts
Now we apply some different cuts and evaluate their behaviour between the signal and background data sets. Note the real data contains only background data as the blinded region is removed. The simulated data contains the Monte-Carlo simulated signal peak.

In [4]:
fts = ['MINIP']
features = [particle + "_" + ft for ft in fts for particle in PARTICLES]

## Begin the Run Below

In [5]:
sdf = sim.fetch_features(["Lb_BKGCAT"] + features)
rdf = real.fetch_features(features)

In [6]:
sdf['Category'] = np.where(sdf['Lb_BKGCAT'].isin([10, 50]), 1, 0) # 1 is a signal event and 0 is a background event
sdf.drop('Lb_BKGCAT', axis=1, inplace=True)
sdf.head()

,p_MINIP,K_MINIP,L1_MINIP,L2_MINIP,Category
eventNumber,,,,,
4826,0.250324,0.298112,0.356679,0.202964,0
4839,0.369802,0.605301,0.244739,0.361498,1
4844,0.101261,0.167225,0.116537,0.351074,1
1686,0.179403,0.478647,0.253145,0.285299,0
1603,0.560972,0.381322,0.556597,0.313683,0


In [31]:
"""for particle in PARTICLES:
    sdf[particle + "_PT"] = np.sqrt(sdf[particle + "_PX"]**2 + sdf[particle + "_PY"]**2)
    sdf[particle + "_P"] = np.sqrt(sdf[particle + "_PT"]**2 + sdf[particle + "_PZ"]**2)
    sdf[particle + "_ThetaCalc"] = np.arcsin(sdf[particle + "_PT"]/sdf[particle + "_P"])
    sdf[particle + "_ETACalc"] = -np.log(np.tan(sdf[particle + "_ThetaCalc"]/2))
    sdf.drop([particle + "_PX", particle + "_PY", particle + "_PZ"], axis=1, inplace=True)
    
    rdf[particle + "_PT"] = np.sqrt(rdf[particle + "_PX"]**2 + rdf[particle + "_PY"]**2)
    rdf[particle + "_P"] = np.sqrt(rdf[particle + "_PT"]**2 + rdf[particle + "_PZ"]**2)
    rdf[particle + "_ThetaCalc"] = np.arcsin(rdf[particle + "_PT"]/rdf[particle + "_P"])
    rdf[particle + "_ETACalc"] = -np.log(np.tan(rdf[particle + "_ThetaCalc"]/2))
    rdf.drop([particle + "_PX", particle + "_PY", particle + "_PZ"], axis=1, inplace=True)"""

In [7]:
g = sdf.groupby('Category')
#features = ['Theta', 'ThetaCalc', 'ETA', 'ETACalc']
#features = [particle + "_" + feature for feature in features for particle in PARTICLES]

In [8]:
fancy_names = {'L1': "$\mu$", 'L2': "e", 'p': "p", 'K': 'K'}

nbins = 100
for variable in features:
    print(f'Starting {variable}')
    feature = variable.split('_')
    fig, ax = plt.subplots(1, 1, figsize=(9, 7))
    hist = ax.hist(g.get_group(1)[variable], label='MC Signal', bins=nbins, histtype='bar', edgecolor='k', alpha=0.6, density=True)
    ax.hist(g.get_group(0)[variable], label='MC Background', bins=hist[1], histtype='bar', edgecolor='k', alpha=0.6, density=True)
    ax.hist(rdf[variable], label='Real Background', bins=hist[1], histtype='bar', edgecolor='k', alpha=0.6, density=True)
    ax.set_ylabel('Normalised Frequency')
    ax.set_xlabel(rf'{feature[1]} of {fancy_names[feature[0]]}')
    plt.legend()
    plt.savefig(f'/home/user211/project/images/Eta/{variable}.png') # Save the figure
    plt.close() # Prevent plot rendering to save compute

Starting p_MINIP
Starting K_MINIP
Starting L1_MINIP
Starting L2_MINIP


## Evaluating Cut Paramters
So we have identified a cut we might care about that is cutting positively and negatively around p_ProbNNp and p_ProbNNk respectively. We need to tune the paramters to optimise these cuts and we do this by plotting some FoM against the paramter being tuned.

In [5]:
def sb_foms(grouped_data):
    s = np.array([len(g.get_group(1)) for g in grouped_data])
    b = np.array([len(g.get_group(0)) for g in grouped_data])
    return s/np.sqrt(b), s/np.sqrt(s+b)

def punzi_fom(original_data, grouped_cut_data, a=5):
    # By default use a 5 sigma CL
    slens = np.array([len(g.get_group(1)) for g in grouped_cut_data]) # Amount of signal events in data after cuts applied
    blens = np.array([len(g.get_group(0)) for g in grouped_cut_data]) # Amount of background events in data after cuts applied
    return (slens/len(original_data.groupby('Category').get_group(1)))/((a/2) + np.sqrt(blens))

In [36]:
def probnnp_cut(data, p):
    d = []
    for prob in p:
        x = data.loc[data[fut] <= prob]
        d.append(x.groupby('Category'))
    return d

ptr = np.linspace(0.2, 0.8, 500)
dab = probnnp_cut(sdf, ptr)

NameError: name 'lower' is not defined

In [ ]:
max_line = ptr[np.argmax(punzi_fom(sdf, dab))]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 7))

ax.scatter(ptr, punzi_fom(sdf, dab), label='Punzi FoM', s=2)
print(max_line)
ax.set_ylabel('Punzi Figure of Merit Value (arb)')
ax.set_xlabel('Probability value to cut data below')
plt.axvline(max_line, c='r', label=f'p={max_line:.4f}')
plt.title(f'Punzi FoM versus value of {fut} cut parameter')
plt.legend()
plt.show()
plt.savefig(f'/home/user211/project/images/punzi_{fut}.png')